In [62]:
from obspy import read
from obspy import UTCDateTime
import plotly.io as pio
pio.renderers.default = 'iframe'
import numpy as np

In [9]:
# load in S086
path_086 = './20221031_M25/086/'
path_JKYD = './20221031_M25/CO.JKYD.00.HH*'

In [10]:
# event origin time
event_otime = UTCDateTime('2022-10-31 01:33:46')

In [54]:
st_086 = read(f"{path_086}/*", starttime=event_otime+1.3, endtime=event_otime+4.0)
st_jkyd = read(f"{path_JKYD}", starttime=event_otime+1.3, endtime=event_otime+4.0)

In [55]:
# preprocess waveforms
st_086.detrend("demean")
st_086.detrend("linear")
st_086.normalize(global_max=True)

st_jkyd.detrend("demean")
st_jkyd.detrend('linear')
st_jkyd.normalize(global_max=True)

# merge them
merged = st_086 + st_jkyd

3 Trace(s) in Stream:
CO.JKYD.00.HHE | 2022-10-31T01:33:47.299999Z - 2022-10-31T01:33:49.999999Z | 100.0 Hz, 271 samples
CO.JKYD.00.HHN | 2022-10-31T01:33:47.299999Z - 2022-10-31T01:33:49.999999Z | 100.0 Hz, 271 samples
CO.JKYD.00.HHZ | 2022-10-31T01:33:47.299999Z - 2022-10-31T01:33:49.999999Z | 100.0 Hz, 271 samples

### Plot waveform

In [137]:
import plotly.graph_objects as go

fig = go.Figure()
y_labels = []

for i, st in enumerate(merged):
    sta_name = st.stats.station
    channel = st.stats.channel
    component = st.stats.component
    t = st.times()
    d = st.data
    if channel == 'DPZ':
        d = d * -1
    if sta_name == '3020823':
        sta_name = 'GT086'
    y_labels.append(f"{sta_name} {component}")
    
    fig.add_trace(go.Scatter(x=t, 
                             y=d+i*2,
                             mode='lines', 
                             line=dict(width=1, color='black'),
                             showlegend=False),
                 )
    
    fig.add_annotation(x=0, y=i*2,
            text=f"{sta_name} {component}",
            showarrow=False,
            yshift=12,
            xshift=3,
            xanchor='left',
            font_size=16)
    
fig.add_hline(y=-2, line_width=1)
    
fig.update_layout(
    xaxis_title='Time (s)',
    yaxis_title='Normalized amplitude',
    title='M2.5 2022-10-31 01:33:46 UTC',
    title_y=0.9,
    title_font_size=25,
    font_size=16,
    width=600,
    height=600,
    template=None,
    yaxis=dict(tickvals=[],zeroline=False),
    #xaxis=dict(zeroline=False)
    
)
fig.show()
fig.write_image('M25.pdf')

### Overlay two waveform

In [136]:
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(x=merged[5].times(), 
                         y=merged[5].data,
                         mode='lines', 
                         line=dict(width=1, color='black'),
                         showlegend=True, name='JKYD'),
                 )

fig.add_trace(go.Scatter(x=merged[2].times(), 
                         y=merged[2].data*-1,
                         mode='lines', 
                         line=dict(width=1, color='red'),
                         showlegend=True,
                        name='GT086'),
                 )

    
fig.add_hline(y=-1.25, line_width=1)
    
fig.update_layout(
    xaxis_title='Time (s)',
    yaxis_title='Normalized amplitude',
    title='M2.5 2022-10-31 01:33:46 UTC',
    title_y=0.9,
    title_font_size=25,
    font_size=16,
    width=600,
    height=350,
    template=None,
    yaxis=dict(tickvals=[],zeroline=False, range=[-1.25, 1.25]),
    legend=dict(x=0.01)
    #xaxis=dict(zeroline=False)
    
)
fig.show()
fig.write_image('M25_wvcomp.pdf')

### Compute and plot spectrum

In [83]:
sts = read('./20221031_M25/for_plot/*.cut')

In [84]:
sts.normalize(global_max=True)

6 Trace(s) in Stream:
.3020823..DPE  | 2022-10-31T01:33:47.664062Z - 2022-10-31T01:33:51.164063Z | 250.0 Hz, 876 samples
.3020823..DPN  | 2022-10-31T01:33:47.664062Z - 2022-10-31T01:33:51.164063Z | 250.0 Hz, 876 samples
.3020823..DPZ  | 2022-10-31T01:33:47.664062Z - 2022-10-31T01:33:51.164063Z | 250.0 Hz, 876 samples
CO.JKYD.00.HHE | 2022-10-31T01:33:47.659997Z - 2022-10-31T01:33:51.159997Z | 100.0 Hz, 351 samples
CO.JKYD.00.HHN | 2022-10-31T01:33:47.659997Z - 2022-10-31T01:33:51.159997Z | 100.0 Hz, 351 samples
CO.JKYD.00.HHZ | 2022-10-31T01:33:47.659997Z - 2022-10-31T01:33:51.159997Z | 100.0 Hz, 351 samples

In [135]:
import obspy
from obspy.signal.konnoohmachismoothing import konno_ohmachi_smoothing
import obspy.signal.util
import numpy as np
from numpy.fft import rfft, rfftfreq
import matplotlib.pyplot as plt

st1 = sts[5]#.normalize()
st2 = sts[2]#.normalize()

spec, freqs = rfft(st1.data), rfftfreq(st1.stats.npts, st1.stats.delta)
spec2, freqs2 = rfft(st2.data), rfftfreq(st2.stats.npts, st2.stats.delta)

fig = go.Figure()


fig.add_trace(go.Scatter(x=freqs, 
                         y=np.abs(spec)/np.abs(spec).max(),
                         mode='lines', 
                         line=dict(width=1, color='black'),
                         showlegend=True, name='JKYD'),
                 )

fig.add_trace(go.Scatter(x=freqs2, 
                         y=np.abs(spec2)/np.abs(spec2).max(),
                         mode='lines', 
                         line=dict(width=1, color='red'),
                         showlegend=True,
                        name='GT086'),
                 )


fig.update_xaxes(type="log")
fig.update_yaxes(type="log")
fig.update_yaxes(dtick=1)
fig.update_xaxes(dtick=1)

#fig.update_layout(yaxis_tickformat = ".e")
fig.update_layout(
    xaxis_title='Frequency (Hz)',
    yaxis_title='Amplitude',
    #title='M2.5 2022-10-31 01:33:46 UTC',
    title_y=0.9,
    title_font_size=25,
    font_size=16,
    width=600,
    height=400,
    template=None,
    yaxis=dict(zeroline=True, showline=True),
    legend=dict(x=0.01),
    xaxis=dict(zeroline=False, showline=True),
    
)



#plt.figure(figsize=(12, 6))
#plt.loglog(freqs, np.abs(spec)/np.abs(spec).max(), label="raw", color="red")
#plt.loglog(freqs2, np.abs(spec2)/np.abs(spec2).max(), label="raw", color="grey")
#plt.loglog(freqs,
#           obspy.signal.util.smooth(abs(spec), 5),
#           label="moving average")
#plt.loglog(freqs,
#           konno_ohmachi_smoothing(abs(spec), freqs, normalize=True),
#           label="konno ohmachi")

#plt.legend()
#plt.show()
fig.show()
fig.write_image('M25_spectrum.pdf')

In [139]:
conda list plotly

# packages in environment at /Users/lindsaychuang/opt/anaconda3/envs/scatseisnet:
#
# Name                    Version                   Build  Channel
plotly                    5.13.1                     py_0    plotly

Note: you may need to restart the kernel to use updated packages.
